In [1]:
from gurobipy import *

## Explication type (1-1)

In [2]:
# Données du problème
data = {'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}

# Identification des ensembles Pros et Cons
pros = [k for k, v in data.items() if v > 0]  # Ensemble Pros(x,y)
cons = [k for k, v in data.items() if v < 0]  # Ensemble Cons(x,y)

print(f"Pros: {pros}")
print(f"Cons: {cons}")

# Calcul des Trade-offs (1-1) valides
valid_pairs = []
for p in pros:
    for q in cons:
        if data[p] + data[q] > 0:
            valid_pairs.append((p, q))

print(f"Nombre de paires valides identifiées : {len(valid_pairs)}")

# Modélisation avec Gurobi

# Instanciation du modèle
m = Model("Explication_TradeOff_1_1")

# Création des variables binaires c[(p,q)]
# c = 1 si la paire est choisie, 0 sinon
c = {}
for p, q in valid_pairs:
    c[(p, q)] = m.addVar(vtype=GRB.BINARY, name=f"c_{p}_{q}")

# Mise à jour du modèle
m.update()

# Ajout des contraintes

# Contrainte 1 : Tout élément de Cons doit être couvert exactement une fois
# Somme(c_pq) = 1 pour chaque q dans Cons
for q in cons:
    # On utilise quicksum pour sommer efficacement les variables Gurobi concernées
    m.addConstr(quicksum(c[(p, q)] for p in pros if (p, q) in valid_pairs) == 1, name=f"Cover_{q}")

# Contrainte 2 : Tout élément de Pros est utilisé au maximum une fois
# Les trade-offs doivent être disjoints
for p in pros:
    m.addConstr(quicksum(c[(p, q)] for q in cons if (p, q) in valid_pairs) <= 1, name=f"Unique_{p}")

# Fonction Objectif
# Ici on cherche juste la faisabilité, on peut minimiser 0
m.setObjective(0, GRB.MINIMIZE)

# Paramétrage (mode silencieux)
m.params.outputflag = 0

# Résolution du PL
m.optimize()

# --- 4. Analyse des résultats ---
if m.status == GRB.OPTIMAL:
    print("\nSolution Optimale Trouvée (Explication Valide) :")
    explanation = []
    for p, q in valid_pairs:
        # On vérifie si la variable est à 1 (avec une petite tolérance pour les flottants)
        if c[(p, q)].x > 0.5:
            explanation.append((p,q))
            net_val = data[p] + data[q]
            print(f" - Trade-off ({p},{q})")
    
    # Vérification
    expected = {('A', 'C'), ('D', 'F'), ('E', 'G')}
    is_same = set(explanation) == expected

elif m.status == GRB.INFEASIBLE:
    print("\nModèle Infeasible : Il n'existe pas d'explication de type (1-1).")
    # Certificat de non-existence

Pros: ['A', 'D', 'E']
Cons: ['C', 'F', 'G']
Nombre de paires valides identifiées : 6
Set parameter Username
Set parameter LicenseID to value 2755283
Academic license - for non-commercial use only - expires 2026-12-15

Solution Optimale Trouvée (Explication Valide) :
 - Trade-off (A,C)
 - Trade-off (D,F)
 - Trade-off (E,G)


### Justification de la Modélisation (1-1)

**Variables de décision :**
*   $c_{pq} \in \{0, 1\}$ : Variable binaire qui vaut 1 si le critère $p \in Pros$ compense le critère $q \in Cons$, 0 sinon.
*   Nous ne considérons que les paires "valides" où la somme des contributions est positive ($\delta_p + \delta_q > 0$) afin de réduire la taille du problème.

**Contraintes :**
1.  **Couverture complète des Cons :** $\sum_{p \in Pros} c_{pq} = 1, \forall q \in Cons$. Chaque désavantage doit être compensé exactement une fois.
2.  **Utilisation unique des Pros :** $\sum_{q \in Cons} c_{pq} \le 1, \forall p \in Pros$. Chaque avantage ne peut servir à compenser qu'un seul désavantage.

**Validité :**
*   La condition de validité ($\delta_p + \delta_q > 0$) est assurée par le pré-filtrage des variables $c_{pq}$ lors de leur création. Si une paire n'est pas valide, la variable n'existe simplement pas dans le modèle.

## Données

In [3]:
# TABLEAU DE DONNÉES COMPLET (Notes des candidats)
candidates_data = {
    'x': {'A': 85, 'B': 81, 'C': 71, 'D': 69, 'E': 75, 'F': 81, 'G': 88},
    'y': {'A': 81, 'B': 81, 'C': 75, 'D': 63, 'E': 67, 'F': 88, 'G': 95},
    'z': {'A': 74, 'B': 89, 'C': 74, 'D': 81, 'E': 68, 'F': 84, 'G': 79},
    't': {'A': 74, 'B': 71, 'C': 84, 'D': 91, 'E': 77, 'F': 76, 'G': 73},
    'u': {'A': 72, 'B': 75, 'C': 66, 'D': 85, 'E': 88, 'F': 66, 'G': 93},
    'v': {'A': 71, 'B': 73, 'C': 63, 'D': 92, 'E': 76, 'F': 79, 'G': 93},
    'w': {'A': 79, 'B': 69, 'C': 78, 'D': 76, 'E': 67, 'F': 84, 'G': 79},
    "w'": {'A': 57, 'B': 76, 'C': 81, 'D': 76, 'E': 82, 'F': 86, 'G': 77}
}

weights = {'A': 8, 'B': 7, 'C': 7, 'D': 6, 'E': 6, 'F': 5, 'G': 6}

def get_notes(candidate_name):
    return candidates_data[candidate_name]

print("Données chargées :")
print("Candidats :", list(candidates_data.keys()))
print("Poids :", weights)


Données chargées :
Candidats : ['x', 'y', 'z', 't', 'u', 'v', 'w', "w'"]
Poids : {'A': 8, 'B': 7, 'C': 7, 'D': 6, 'E': 6, 'F': 5, 'G': 6}


## Explication type (1-m)

In [4]:
def compare_1_m(candidat_1, candidat_2):

    # DONNÉES (Comparaison u > v)
    notes_1 = get_notes(candidat_1)
    notes_2 = get_notes(candidat_2)

    # Calcul des contributions (deltas)
    deltas = {}
    for k in weights:
        deltas[k] = weights[k] * (notes_1[k] - notes_2[k])

    # Calcul des contributions (deltas)
    deltas = {}
    for k in weights:
        deltas[k] = weights[k] * (notes_1[k] - notes_2[k])

    pros = [k for k, v in deltas.items() if v > 0]
    cons = [k for k, v in deltas.items() if v < 0]

    print(f"Comparaison {candidat_1} > {candidat_2}")
    print(f"Arguments Pour (Pros) : {pros}")
    print(f"Arguments Contre (Cons) : {cons}")
    print(f"Deltas : {deltas}")

    # MODÉLISATION GUROBI
    m = Model("Explication_1_m")

    # Variables de décision UNIQUEMENT x
    # x[p, c] = 1 si l'argument Pro 'p' couvre l'argument Con 'c'
    x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

    m.update()

    # CONTRAINTES

    # A. Couverture complète : Chaque Con doit être couvert par exactement un Pro
    for c in cons:
        m.addConstr(quicksum(x[p, c] for p in pros) == 1, name=f"Cover_{c}")

    # B. Validité du Trade-off
    # Formule : Delta_p + Somme(Delta_c * x_pc) >= epsilon
    # Si p est utilisé, cela vérifie que le trade-off est positif.
    # Si p est INUTILISÉ (x_pc = 0 partout), cela devient Delta_p >= epsilon, ce qui est toujours VRAI.
    epsilon = 0.001 

    for p in pros:
        # Note: deltas[c] est négatif ici
        m.addConstr(deltas[p] + quicksum(deltas[c] * x[p, c] for c in cons) >= epsilon, name=f"Valid_{p}")

    # RÉSOLUTION
    m.params.outputflag = 0  
    m.optimize()

    # ANALYSE
    print("-" * 30)
    if m.status == GRB.OPTIMAL:
        print("Résultat : Une explication de type (1-m) existe !\n")
        
        # Pour chaque Con, on affiche les Pros qui le couvrent
        for c in cons:
            assigned_pros = [p for p in pros if x[p, c].x > 0.5]
            
            sum_pros = sum(deltas[p] for p in assigned_pros)
            net_value = deltas[c] + sum_pros
            
            print(f"Argument Contre {c} ({deltas[c]}) compensé par :")
            print(f"  -> Pros : {assigned_pros} (Cumul Pros: +{sum_pros})")
            print(f"  -> Bilan net : +{net_value}")
            print("")

    elif m.status == GRB.INFEASIBLE:
        print("Résultat : INFEASIBLE (Certificat de non-existence).")


compare_1_m("x", "y")

Comparaison x > y
Arguments Pour (Pros) : ['A', 'D', 'E']
Arguments Contre (Cons) : ['C', 'F', 'G']
Deltas : {'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}
------------------------------
Résultat : Une explication de type (1-m) existe !

Argument Contre C (-28) compensé par :
  -> Pros : ['A'] (Cumul Pros: +32)
  -> Bilan net : +4

Argument Contre F (-35) compensé par :
  -> Pros : ['D'] (Cumul Pros: +36)
  -> Bilan net : +1

Argument Contre G (-42) compensé par :
  -> Pros : ['E'] (Cumul Pros: +48)
  -> Bilan net : +6



### Justification de la Modélisation (1-m)

**Variables de décision :**
*   $x_{pc} \in \{0, 1\}$ : Vaut 1 si l'argument *Pro* $p$ couvre l'argument *Con* $c$.

**Contraintes :**
1.  **Couverture des Cons :** $\sum_{p \in Pros} x_{pc} = 1, \forall c \in Cons$. Chaque *Con* doit être couvert par exactement un *Pro*. Un *Pro* peut couvrir plusieurs *Cons*.
2.  **Validité du Trade-off (1-m) :** Pour chaque *Pro* $p$, la somme de sa contribution positive et des contributions négatives qu'il couvre doit rester positive :
    $$ \delta_p + \sum_{c \in Cons} \delta_c x_{pc} \ge \epsilon $$
    Cette contrainte n'est active que si $p$ est utilisé. Si $p$ n'est pas utilisé (tous les $x_{pc}=0$), la contrainte devient $\delta_p \ge \epsilon$, ce qui est trivialement vrai car $\delta_p > 0$.

**Choix de $\epsilon$ :**
*   On utilise un petit $\epsilon = 0.001$ pour simuler une inégalité stricte ($> 0$).

## Comparaison u > v

In [5]:
compare_1_m("u", "v")

Comparaison u > v
Arguments Pour (Pros) : ['A', 'B', 'C', 'E']
Arguments Contre (Cons) : ['D', 'F']
Deltas : {'A': 8, 'B': 14, 'C': 21, 'D': -42, 'E': 72, 'F': -65, 'G': 0}
------------------------------
Résultat : INFEASIBLE (Certificat de non-existence).


## Explication type (m-1)

On cherche ici à expliquer la préférence $x \succ y$ à l'aide de trade-offs de type (m-1).
Un trade-off de type (m-1) est constitué d'un seul argument *Con* $c$ et d'un ensemble d'arguments *Pros* $\mathcal{P}'$ qui le compensent, c'est-à-dire :
$$ \delta_c + \sum_{p \in \mathcal{P}'} \delta_p > 0 $$

L'explication doit couvrir tous les arguments *Cons* de manière disjointe (chaque *Pro* ne sert qu'une fois).

In [ ]:
def compare_m_1(candidat_1, candidat_2):

    # DONNÉES (Comparaison 1 > 2)
    notes_1 = get_notes(candidat_1)
    notes_2 = get_notes(candidat_2)

    # Calcul des contributions (deltas)
    deltas = {}
    for k in weights:
        deltas[k] = weights[k] * (notes_1[k] - notes_2[k])

    pros = [k for k, v in deltas.items() if v > 0]
    cons = [k for k, v in deltas.items() if v < 0]

    print(f"Comparaison {candidat_1} > {candidat_2}")
    print(f"Arguments Pour (Pros) : {pros}")
    print(f"Arguments Contre (Cons) : {cons}")
    print(f"Deltas : {deltas}")

    # MODÉLISATION GUROBI (m-1)
    m = Model("Explication_m_1")

    # Variables de décision x[p, c]
    # x[p, c] = 1 si l'argument Pro 'p' est utilisé pour couvrir l'argument Con 'c'
    x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

    m.update()

    # CONTRAINTES

    # 1. Disjonction des Pros : Chaque Pro peut être utilisé au plus une fois
    for p in pros:
        m.addConstr(quicksum(x[p, c] for c in cons) <= 1, name=f"Unique_{p}")

    # 2. Validité des Trade-offs : Chaque Con 'c' doit être compensé par un groupe de Pros
    # Formule : Delta_c + Somme(Delta_p * x_pc) >= epsilon
    epsilon = 0.001

    for c in cons:
        # Note: deltas[c] est négatif
        m.addConstr(deltas[c] + quicksum(deltas[p] * x[p, c] for p in pros) >= epsilon, name=f"Valid_Group_for_{c}")

    # Fonction Objectif : Ici on cherche juste la faisabilité (Objectif constant)
    m.setObjective(0, GRB.MINIMIZE)

    # Paramétrage
    m.params.outputflag = 0
    m.optimize()

    # ANALYSE DES RÉSULTATS
    print("-" * 30)
    if m.status == GRB.OPTIMAL:
        print("Résultat : Une explication de type (m-1) existe !\n")
        
        # Pour chaque Con, on affiche les Pros qui le couvrent
        for c in cons:
            assigned_pros = [p for p in pros if x[p, c].x > 0.5]
            
            sum_pros = sum(deltas[p] for p in assigned_pros)
            net_value = deltas[c] + sum_pros
            
            print(f"Argument Contre {c} ({deltas[c]}) compensé par :")
            print(f"  -> Pros : {assigned_pros} (Cumul Pros: +{sum_pros})")
            print(f"  -> Bilan net : +{net_value}")
            print("")

    elif m.status == GRB.INFEASIBLE:
        print("Résultat : INFEASIBLE (Pas d'explication de type (m-1)).")


compare_m_1("x", "y")


Comparaison x > y
Arguments Pour (Pros) : ['A', 'D', 'E']
Arguments Contre (Cons) : ['C', 'F', 'G']
Deltas : {'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}
------------------------------
Résultat : Une explication de type (m-1) existe !

Argument Contre C (-28) compensé par :
  -> Pros : ['A'] (Cumul Pros: +32)
  -> Bilan net : +4

Argument Contre F (-35) compensé par :
  -> Pros : ['D'] (Cumul Pros: +36)
  -> Bilan net : +1

Argument Contre G (-42) compensé par :
  -> Pros : ['E'] (Cumul Pros: +48)
  -> Bilan net : +6



### Justification de la Modélisation (m-1)

**Variables de décision :**
*   $x_{pc} \in \{0, 1\}$ : Vaut 1 si l'argument *Pro* $p$ participe à la couverture de l'argument *Con* $c$.

**Contraintes :**
1.  **Utilisation unique des Pros :** $\sum_{c \in Cons} x_{pc} \le 1, \forall p \in Pros$. Chaque *Pro* ne peut participer qu'à un seul groupe de compensation.
2.  **Validité du Trade-off (m-1) :** Pour chaque *Con* $c$, la somme de sa contribution négative et des contributions des *Pros* qui le couvrent doit être positive :
    $$ \delta_c + \sum_{p \in Pros} \delta_p x_{pc} \ge \epsilon $$
    Contrairement au cas (1-m), chaque *Con* doit obligatoirement être couvert, donc cette contrainte doit être valide pour tout $c$.

## Comparaison z > t

In [7]:
compare_m_1("z", "t")

Comparaison z > t
Arguments Pour (Pros) : ['B', 'F', 'G']
Arguments Contre (Cons) : ['C', 'D', 'E']
Deltas : {'A': 0, 'B': 126, 'C': -70, 'D': -60, 'E': -54, 'F': 40, 'G': 36}
------------------------------
Résultat : INFEASIBLE (Pas d'explication de type (m-1)).


## Explication Mixte (Question 4)

On cherche maintenant une explication qui combine des trade-offs de type (1-m) et (m-1) pour former une explication complète de la préférence $x \succ y$.

In [8]:
def mixt_compare(candidat_1, candidat_2, notes_1=None, notes_2=None):
    
    # DONNÉES Comparaison 1 > 2
    if notes_1 is None:
        notes_1 = get_notes(candidat_1)
    if notes_2 is None:
        notes_2 = get_notes(candidat_2)

    # Calcul des contributions
    deltas = {}
    for k in weights:
        deltas[k] = weights[k] * (notes_1[k] - notes_2[k])

    pros = [k for k, v in deltas.items() if v > 0]
    cons = [k for k, v in deltas.items() if v < 0]

    print(f"Comparaison {candidat_1} > {candidat_2}")
    print(f"Arguments Pour (Pros) : {pros}")
    print(f"Arguments Contre (Cons) : {cons}")
    print(f"Deltas : {deltas}")

    # MODÈLE MIXTE
    m = Model("Explication_Mixte")

    # Variables
    z_1m = m.addVars(pros, vtype=GRB.BINARY, name="z_1m")  # p est pivot 1-m
    z_m1 = m.addVars(cons, vtype=GRB.BINARY, name="z_m1")  # c est pivot m-1

    v = m.addVars(pros, cons, vtype=GRB.BINARY, name="v") # relation dans 1-m (p couvre c)
    w = m.addVars(pros, cons, vtype=GRB.BINARY, name="w") # relation dans m-1 (p aide c)

    m.update()

    # CONTRAINTES

    # 1. Couverture des Cons
    # Chaque c est soit un pivot m-1, soit couvert par un p en mode 1-m
    for c in cons:
        m.addConstr(z_m1[c] + quicksum(v[p, c] for p in pros) == 1, name=f"CoverCons_{c}")

    # 2. Utilisation unique des Pros
    # Chaque p est soit un pivot 1-m, soit un support m-1 (ou rien)
    for p in pros:
        m.addConstr(z_1m[p] + quicksum(w[p, c] for c in cons) <= 1, name=f"UniquePro_{p}")

    # 3. Consistance des liens
    # Si p couvre c en 1-m (v[p,c]=1), alors p doit être un pivot 1-m
    for p in pros:
        for c in cons:
            m.addConstr(v[p, c] <= z_1m[p])

    # Si p aide c en m-1 (w[p,c]=1), alors c doit être un pivot m-1
    for p in pros:
        for c in cons:
            m.addConstr(w[p, c] <= z_m1[c])

    # 4. Validité des groupes (Solvabilité budgétaire)
    epsilon = 0.001
    BigM = 1000

    # Pour chaque p pivot 1-m : Delta_p + Somme(Delta_c) > 0
    for p in pros:
        # Si z_1m[p] = 0, la contrainte est relâchée ( >= -1000 )
        m.addConstr(deltas[p] + quicksum(deltas[c] * v[p, c] for c in cons) >= epsilon - BigM * (1 - z_1m[p]))

    # Pour chaque c pivot m-1 : Delta_c + Somme(Delta_p) > 0
    for c in cons:
        # Si z_m1[c] = 0, la contrainte est relâchée
        m.addConstr(deltas[c] + quicksum(deltas[p] * w[p, c] for p in pros) >= epsilon - BigM * (1 - z_m1[c]))

    # Objectif : Minimiser le nombre de pivots (pour avoir une explication compacte) ou juste Feasibility
    # Essayons de minimiser le nombre total de trade-offs
    m.setObjective(quicksum(z_1m[p] for p in pros) + quicksum(z_m1[c] for c in cons), GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    # RÉSULTATS
    print("-" * 30)
    if m.status == GRB.OPTIMAL:
        print("Solution Mixte trouvée !\n")
        
        # Affichage des trade-offs (1-m)
        for p in pros:
            if z_1m[p].x > 0.5:
                my_cons = [c for c in cons if v[p, c].x > 0.5]
                val = deltas[p] + sum(deltas[c] for c in my_cons)
                print(f"[Type 1-m] Le Pro '{p}' ({deltas[p]}) couvre les Cons {my_cons} (Net: +{val})")

        # Affichage des trade-offs (m-1)
        for c in cons:
            if z_m1[c].x > 0.5:
                my_pros = [p for p in pros if w[p, c].x > 0.5]
                val = deltas[c] + sum(deltas[p] for p in my_pros)
                print(f"[Type m-1] Le Con '{c}' ({deltas[c]}) est compensé par les Pros {my_pros} (Net: +{val})")
                
    elif m.status == GRB.INFEASIBLE:
        print("INFEASIBLE. Pas d'explication mixte possible.")

mixt_compare("x","y")


Comparaison x > y
Arguments Pour (Pros) : ['A', 'D', 'E']
Arguments Contre (Cons) : ['C', 'F', 'G']
Deltas : {'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}
------------------------------
Solution Mixte trouvée !

[Type 1-m] Le Pro 'A' (32) couvre les Cons ['C'] (Net: +4)
[Type m-1] Le Con 'F' (-35) est compensé par les Pros ['D'] (Net: +1)
[Type m-1] Le Con 'G' (-42) est compensé par les Pros ['E'] (Net: +6)


In [9]:
# NOUVEAUX CANDIDATS (a1>a2)
notes_a1 = {'A': 89, 'B': 74, 'C': 81, 'D': 68, 'E': 84, 'F': 79, 'G': 77}
notes_a2 = {'A': 71, 'B': 84, 'C': 91, 'D': 79, 'E': 78, 'F': 73.5, 'G': 77}

mixt_compare("a1", "a2", notes_a1, notes_a2)

Comparaison a1 > a2
Arguments Pour (Pros) : ['A', 'E', 'F']
Arguments Contre (Cons) : ['B', 'C', 'D']
Deltas : {'A': 144, 'B': -70, 'C': -70, 'D': -66, 'E': 36, 'F': 27.5, 'G': 0}
------------------------------
INFEASIBLE. Pas d'explication mixte possible.


### Justification de la Modélisation Mixte

Pour combiner les types (1-m) et (m-1), nous devons décider pour chaque élément s'il joue le rôle de pivot (chef de groupe) ou de membre de soutien.

**Variables de décision :**
*   **Pivots** :
    *   $z^{1m}_p \in \{0,1\}$ : Le *Pro* $p$ mène un trade-off (1-m).
    *   $z^{m1}_c \in \{0,1\}$ : Le *Con* $c$ est au cœur d'un trade-off (m-1).
*   **Liens** :
    *   $v_{pc} \in \{0,1\}$ : $p$ couvre $c$ (dans un motif 1-m).
    *   $w_{pc} \in \{0,1\}$ : $p$ aide à compenser $c$ (dans un motif m-1).

**Contraintes Fondamentales :**
1.  **Partition des Cons :** Chaque *Con* $c$ doit être soit un pivot de type (m-1), soit être couvert par un *Pro* (type 1-m). Les deux cas sont exclusifs :
    $$ z^{m1}_c + \sum_{p} v_{pc} = 1 $$
2.  **Partition des Pros :** Chaque *Pro* $p$ doit être soit un pivot de type (1-m), soit aider un *Con* (type m-1), soit être inutilisé :
    $$ z^{1m}_p + \sum_{c} w_{pc} \le 1 $$

**Contraintes Logiques (Big-M) :**
*   L'objectif est d'imposer la règle **"Le bilan du groupe doit être positif"** UNIQUEMENT si le groupe **existe** (c'est-à-dire si le pivot est actif).
*   Pour cela, nous utilisons la méthode "Big-M" qui consiste à ajouter un terme de relâchement très grand ($M$) lorsque la condition n'est pas remplie.
*   Par exemple, pour un pivot (1-m) potentiel $p$, la contrainte est :
    $$ \underbrace{\delta_p + \sum_{c} \delta_c v_{pc}}_{\text{Bilan du groupe}} \ge \epsilon - \underbrace{M(1 - z^{1m}_p)}_{\text{Terme "Joker"}} $$
*   **Cas 1 : Pivot Actif ($z^{1m}_p=1$)**
    *   Le terme "Joker" devient $M(1-1) = 0$.
    *   La contrainte devient $\text{Bilan} \ge \epsilon$. $\rightarrow$ **La règle de validité est STRICTE et ACTIVE.**
*   **Cas 2 : Pivot Inactif ($z^{1m}_p=0$)**
    *   Le terme "Joker" devient $M(1-0) = M$ (ex: 1000).
    *   La contrainte devient $\text{Bilan} \ge \epsilon - 1000$ (ex: $\ge -999.999$).
    *   Comme le "Bilan" (somme des notes) ne descendra jamais aussi bas, cette contrainte est mathématiquement **TOUJOURS VRAIE**. Elle ne contraint plus rien. $\rightarrow$ **La règle est DÉSACTIVÉE, le solveur est libre.**